In [5]:
import porepy as pp
import numpy as np
import scipy.sparse as sps

from time import time

from buoyancy_flow_model import ModelGeometry2D, ModelGeometry3D
from buoyancy_flow_model import (
    ModelMDGeometry2D,
    ModelMDGeometry3D,
)
from buoyancy_flow_model import (
    BuoyancyFlowModel2N,
    BuoyancyFlowModel3N,
)
from buoyancy_flow_model import to_Mega


# Parameterization list for both tests
Parameterization = [
    (BuoyancyFlowModel2N, True, 4),
    (BuoyancyFlowModel2N, False, 4),
    (BuoyancyFlowModel3N, True, 4),
    (BuoyancyFlowModel3N, False, 4),
]


def _run_buoyancy_model(
    model_class: type,
    mesh_2d_Q: bool,
    expected_order_loss: int,
    md: bool = False,
    do_run = True,
) -> None:
    """Run buoyancy flow simulation for given parameters."""
    residual_tolerance = 10.0 ** (-expected_order_loss)
    day = 86400
    if md:
        tf = 0.5 * day
        dt = 0.25 * day
        geometry2d = ModelMDGeometry2D
        geometry3d = ModelMDGeometry3D
    else:
        tf = 2.0 * day
        dt = 1.0 * day
        geometry2d = ModelGeometry2D
        geometry3d = ModelGeometry3D

    solid_constants = pp.SolidConstants(
        permeability=1.0e-14,
        porosity=0.1,
        thermal_conductivity=2.0 * to_Mega,
        density=2500.0,
        specific_heat_capacity=1000.0 * to_Mega,
    )
    time_manager = pp.TimeManager(
        schedule=[0.0, tf],
        dt_init=dt,
        constant_dt=True,
        iter_max=50,
        print_info=True,
    )
    params = {
        "fractional_flow": True,
        "enable_buoyancy_effects": True,
        "material_constants": {"solid": solid_constants},
        "time_manager": time_manager,
        "apply_schur_complement_reduction": False,
        "nl_convergence_tol": np.inf,
        "nl_convergence_tol_res": residual_tolerance,
        "max_iterations": 50,
        "expected_order_loss": expected_order_loss,
    }
    # Combine geometry with model class
    if mesh_2d_Q:

        class Model2D(geometry2d, model_class):
            pass

        model = Model2D(params)
    else:

        class Model3D(geometry3d, model_class):
            pass

        model = Model3D(params)
    if do_run:
        pp.run_time_dependent_model(model, params)
    return model



In [30]:
tic = time()
model = _run_buoyancy_model(BuoyancyFlowModel3N, True, 2, md=True, do_run = True)  # type: ignore
print("Time elapsed test_buoyancy_flow.py:", time() - tic)


eqs = model.equation_system


Time elapsed test_buoyancy_flow.py: 41.221171379089355


In [31]:
num_iters = 10
tot_time = 0.0
for key in eqs.equations.keys():
    tic = time()
    for _ in range(num_iters):
        eqs.evaluate(eqs.equations[key])
    toc = time()
    tot_time += (toc - tic)/num_iters
    print(f"Time elapsed {key}: {(toc - tic)/num_iters}")

print(f"Total time elapsed for {num_iters} iterations: {tot_time}")

Time elapsed mass_balance_equation: 0.016039299964904784
Time elapsed interface_darcy_flux_equation: 0.014029192924499511
Time elapsed well_flux_equation: 0.000751042366027832
Time elapsed component_mass_balance_equation_C5H12: 0.2024096965789795
Time elapsed component_mass_balance_equation_CH4: 0.18585705757141113
Time elapsed energy_balance_equation: 0.18183772563934325
Time elapsed interface_fourier_flux_equation: 0.0038093090057373046
Time elapsed interface_enthalpy_flux_equation: 0.00901634693145752
Time elapsed well_enthalpy_flux_equation: 0.0007307291030883789
Time elapsed elimination_of_s_oil_on_grids_[304, 305, 306, 307, 308, 309, 310, 311, 312, 313]: 0.00039136409759521484
Time elapsed elimination_of_s_gas_on_grids_[304, 305, 306, 307, 308, 309, 310, 311, 312, 313]: 0.00038347244262695315
Time elapsed elimination_of_x_C5H12_water_on_grids_[304, 305, 306, 307, 308, 309, 310, 311, 312, 313]: 0.000376129150390625
Time elapsed elimination_of_x_CH4_water_on_grids_[304, 305, 306, 3

In [32]:
domains = model.mdg.subdomains()

methods = [model.advection_weight_energy_balance(domains),
           model.enthalpy_buoyancy(domains)]

for method in methods:
    tic = time()
    for _ in range(num_iters):
        eqs.evaluate(method)
    toc = time()
    print(f"Time elapsed {method.name}: {(toc - tic)/num_iters}")


for component in model.fluid.components:
    tic = time()
    for _ in range(num_iters):
        eqs.evaluate(model.component_buoyancy(component, domains))
    toc = time()
    print(f"Time elapsed buoyancy {component.name}: {(toc - tic)/num_iters}")
           


Time elapsed advected_enthalpy: 0.00434105396270752
Time elapsed enthalpy_buoyancy: 0.08195774555206299
Time elapsed buoyancy H2O: 0.28471598625183103
Time elapsed buoyancy C5H12: 0.274661660194397
Time elapsed buoyancy CH4: 0.27577354907989504


In [51]:
sz = 10000
x = np.random.rand(sz)
y = np.random.rand(sz)

A = sps.random(sz, sz, density=0.01, format="csr")


x_ad, y_ad = pp.ad.initAdArrays([x, y])

tic = time()
for _ in range(num_iters):
    z = x_ad * y_ad
    z = A @ z + x_ad
toc = time()
print(f"Time elapsed AdArray addition: {(toc - tic)/num_iters}")

a = np.vstack([x, y])
b = np.vstack([y, x])

tic = time()
for _ in range(num_iters):
    z = a * b
    z = A @ x + z[0]
toc = time()
print(f"Time elapsed numpy array addition: {(toc - tic)/num_iters}")



Time elapsed AdArray addition: 0.04717597961425781
Time elapsed numpy array addition: 0.0014455318450927734


In [38]:
sz = 1000000
a = np.random.rand(5, sz)
b = np.random.rand(5, sz)

tic = time()
for _ in range(100):
    c = a + b
toc = time()
print(f"Time elapsed for a + b: {(toc - tic)/100}")

tic = time()
for _ in range(100):
    a += b
toc = time()
print(f"Time elapsed for a += b: {(toc - tic)/100}")



Time elapsed for a + b: 0.008311796188354491
Time elapsed for a += b: 0.004838833808898926
